In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Run VAE models systematically

## Imports

In [24]:
import pandas as pd
import numpy as np
import itertools
import os
from evoscaper.scripts.cvae_scan import main as cvae_scan
from evoscaper.utils.preprocess import make_datetime_str

## Create table of all VAE model training settings

Parameters for:
- Biological dataset generation
- Training data
    - Input
    - Output 
- Model architecture
- Training hyperparameters

### Initial parameters

In [25]:
data_dir = '../data'

hpos_architecture = {
    'seed_arch': 1,
    'hidden_size': 32,
    'enc_layers': [64, 64, 64],
    'dec_layers': [64, 64, 64],
    'model': 'CVAE',
    'use_sigmoid_decoder': False,
    'enc_init': 'HeNormal',
    'dec_init': 'HeNormal',
    'init_model_with_random': True,
    'activation': 'leaky_relu',
}


hpos_training = {
    'seed_train': 1,
    'batch_size': 128,
    'epochs': 2000,
    'patience': 1000,
    'learning_rate': 1e-1,
    'loss_func': 'mse',
    'use_dropout': False,
    'dropout_rate': 0.1,
    'use_l2_reg': False,
    'l2_reg_alpha': 0.01,
    'use_kl_div': True,
    'kl_weight': 0.00025,  # inspired by https://github.com/elttaes/VAE-MNIST-Haiku-Jax/blob/main/cVAE_mnist.ipynb
}
hpos_training['print_every'] = hpos_training['epochs'] // 100

hpos_optimization = {
    'seed_opt': 1,
    'opt_method': 'adam',
    'opt_min_lr': 1e-6,
    'opt_min_delta': 1e-4,
    'learning_rate_sched': 'cosine_decay',
    'use_warmup': True,
    'warmup_epochs': 20,
}

hpos_dataset = {
    'seed_dataset': 1,
    'include_diffs': False,
    'objective_col': 'Log sensitivity',
    'output_species': ['RNA_2'],
    'signal_species': ['RNA_0'],
    'filenames_train_config': [f'{data_dir}/raw/summarise_simulation/2024_12_05_210221/ensemble_config.json'], 
    'filenames_train_table': [f'{data_dir}/raw/summarise_simulation/2024_12_05_210221/tabulated_mutation_info.csv'],
    'filenames_verify_config': [f'{data_dir}/raw/summarise_simulation/2024_11_21_160955/ensemble_config.json'], 
    'filenames_verify_table': [f'{data_dir}/raw/summarise_simulation/2024_11_21_160955/tabulated_mutation_info.csv'],
    'use_test_data': False,
    # 'total_ds': None,   # TO BE RECORDED
    'total_ds_max': 3e6,
    'train_split': 0.8,
    'x_type': 'energies',
    # XY filtering:
    'filt_x_nans': True,
    'filt_y_nans': True,
    'filt_sensitivity_nans': True,
    'filt_precision_nans': True,
    'filt_n_same_x_max': 1,
    'filt_n_same_x_max_bins': 15,
    # XY preprocessing:
    'prep_x_standardise': False,
    'prep_y_standardise': False,
    'prep_x_min_max': False,
    'prep_y_min_max': False,
    'prep_x_robust_scaling': True,
    'prep_y_robust_scaling': True,
    'prep_x_logscale': False,
    'prep_y_logscale': False,
    'prep_x_categorical': False,
    'prep_y_categorical': True,
    'prep_x_categorical_onehot': False,
    'prep_y_categorical_onehot': True,
    'prep_x_categorical_n_bins': 10,
    'prep_y_categorical_n_bins': 10,
    'prep_x_categorical_method': 'quantile',
    'prep_y_categorical_method': 'quantile',
    'prep_x_negative': True,
    'prep_y_negative': False
}

hpos_biological = {
    'n_species': 3,
    'sequence_length': 20,
    'signal_function': 'step_function',
    'signal_target': 2,
    'starting_copynumbers_input': [200],
    'starting_copynumbers_output': [200],
    'starting_copynumbers_other': [200],
    'association_binding_rate': 1000000,
    'include_prod_deg': False,
}

hpos_eval = {
    'eval_n_to_sample': 1e5
}

info_to_be_recorded = {
    'filename_saved_model': 'TO_BE_RECORDED',
    'total_ds': 'TO_BE_RECORDED',
    'n_batches': 'TO_BE_RECORDED',
    'R2_train': 'TO_BE_RECORDED',
    'R2_test': 'TO_BE_RECORDED',
    'mutual_information_conditionality': 'TO_BE_RECORDED',
    'n_layers_enc': 'TO_BE_RECORDED',
    'n_layers_dec': 'TO_BE_RECORDED',
    'run_successful': 'TO_BE_RECORDED',
}



In [26]:
df_hpos = pd.concat([pd.DataFrame.from_dict(hpos, orient='index').T for hpos in [hpos_architecture, hpos_training, hpos_optimization, hpos_dataset, hpos_eval]], axis=1)
assert df_hpos.columns.duplicated().sum() == 0, 'Change some column names, there are duplicates'
basic_setting = df_hpos.copy()
df_hpos

,seed_arch,hidden_size,enc_layers,dec_layers,model,use_sigmoid_decoder,enc_init,dec_init,init_model_with_random,activation,...,prep_y_categorical,prep_x_categorical_onehot,prep_y_categorical_onehot,prep_x_categorical_n_bins,prep_y_categorical_n_bins,prep_x_categorical_method,prep_y_categorical_method,prep_x_negative,prep_y_negative,eval_n_to_sample
0,1,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,True,leaky_relu,...,True,False,True,10,10,quantile,quantile,True,False,100000.0


### All parameters

In [27]:
hpos_to_vary_from_og = {
    'seed_arch': [1, 2, 3, 4, 5],
}
hpos_to_vary_together = {
    'total_ds_max': [1e4, 5e4, 1e5],
    'hidden_size': [16, 32, 64, 128, 256, 512],
}
hpos_to_vary_together2 = {
    'objective_col': ['adaptability', 'sensitivity_wrt_species-6'],
    'x_type': ['energies', 'binding_rates_dissociation'],
    'learning_rate': [1e-2, 1e-3, 1e-4],
    'use_l2_reg': [True],
    'l2_reg_alpha': [0, 1e-2, 1e-3, 1e-4],
    'kl_weight': [1e-4, 1e-3, 1e-2, 1e-1, 1],
}

df_hpos.loc[df_hpos['objective_col'] == 'sensitivity_wrt_species-6', 'prep_y_logscale'] = True

In [28]:
for h, v in hpos_to_vary_from_og.items():
    df_hpos = pd.concat([df_hpos] + [basic_setting.assign(**{h: vv}) for vv in v], ignore_index=True)
df_hpos

,seed_arch,hidden_size,enc_layers,dec_layers,model,use_sigmoid_decoder,enc_init,dec_init,init_model_with_random,activation,...,prep_y_categorical,prep_x_categorical_onehot,prep_y_categorical_onehot,prep_x_categorical_n_bins,prep_y_categorical_n_bins,prep_x_categorical_method,prep_y_categorical_method,prep_x_negative,prep_y_negative,eval_n_to_sample
0,1,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,True,leaky_relu,...,True,False,True,10,10,quantile,quantile,True,False,100000.0
1,1,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,True,leaky_relu,...,True,False,True,10,10,quantile,quantile,True,False,100000.0
2,2,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,True,leaky_relu,...,True,False,True,10,10,quantile,quantile,True,False,100000.0
3,3,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,True,leaky_relu,...,True,False,True,10,10,quantile,quantile,True,False,100000.0
4,4,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,True,leaky_relu,...,True,False,True,10,10,quantile,quantile,True,False,100000.0
5,5,32,"[64, 64, 64]","[64, 64, 64]",CVAE,False,HeNormal,HeNormal,True,leaky_relu,...,True,False,True,10,10,quantile,quantile,True,False,100000.0


In [29]:
def add_combinatorial_keys(df_hpos, hpos_to_vary_together, basic_setting):
    keys_vary_together = sorted(hpos_to_vary_together.keys())
    for v in itertools.product(*[hpos_to_vary_together[h] for h in keys_vary_together]):
        curr = basic_setting.assign(**{h: vv for h, vv in zip(keys_vary_together, v)})
        df_hpos = pd.concat([df_hpos, curr], ignore_index=True)
    print('All good if these are equal: ', len(df_hpos), len(list(itertools.product(*[hpos_to_vary_together[h] for h in keys_vary_together]))) + np.sum([len(v) for v in hpos_to_vary_from_og.values()]) + 1)
    return df_hpos

df_hpos = add_combinatorial_keys(df_hpos, hpos_to_vary_together, basic_setting)
df_hpos = add_combinatorial_keys(df_hpos, hpos_to_vary_together2, basic_setting)

len(df_hpos)

All good if these are equal:  24 24
All good if these are equal:  264 246


264

# Use table to create dataset for training

In [ ]:
# fn = '../data/raw/summarise_simulation/2024_11_21_144918/tabulated_mutation_info.csv'
# # fn = '../data/raw/summarise_simulation/2024_11_21_160955/tabulated_mutation_info.csv'
# # fn = '../data/raw/summarise_simulation/2024_12_05_210221/tabulated_mutation_info.csv'
# data = pd.read_csv(fn).iloc[:100]
# len(data)

100

In [ ]:
# hpos = df_hpos.reset_index().iloc[0]
# cvae_scan(hpos, top_dir=os.path.join('data', make_datetime_str()))


In [ ]:
# REMINDER: Total ds is 3 and the test data is not being used
# cvae_scan(hpos, top_dir=os.path.join('data', make_datetime_str()))

top_dir = os.path.join('data', make_datetime_str())
for i in range(len(df_hpos)):
    hpos = df_hpos.reset_index().iloc[i]
    try:
        hpos = cvae_scan(hpos, top_dir=top_dir)
        hpos['run_successful'] = True
    except:
        hpos['run_successful'] = False
    df_hpos.loc[i] = pd.Series(hpos) if type(hpos) == dict else hpos
    df_hpos.to_csv(os.path.join(top_dir, 'df_hpos.csv'))

# Set up model

In [ ]:
# model_fn = partial(VAE_fn, enc_layers=enc_layers, dec_layers=dec_layers, decoder_head=x.shape[-1], 
#                    HIDDEN_SIZE=HIDDEN_SIZE, decoder_activation_final=jax.nn.sigmoid if USE_SIGMOID_DECODER else jax.nn.leaky_relu, 
#                    enc_init=ENC_INIT, dec_init=DEC_INIT, activation=get_activation_fn(ACTIVATION))
# model_t = hk.multi_transform(model_fn)
# dummy_x = jax.random.normal(PRNG, x.shape)
# dummy_cond = jax.random.normal(PRNG, cond.shape)
# params = model_t.init(PRNG, dummy_x, dummy_cond, deterministic=False)
